<a href="https://colab.research.google.com/github/XuyangAbert/PyGCL/blob/main/PyGCL_Prac_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/XuyangAbert/PyGCL.git

In [ ]:
# !pip install torch==2.1.2
# !pip install torch-geometric==2.2.0
!pip install torch
!pip install torch-geometric
!pip install numpy==1.24.3
# !pip install numpy
!pip install tqdm
!pip install scipy
!pip install networkx
!pip install scikit-learn
!pip install dgl==1.0.0

In [ ]:
!pip install torch_scatter
!pip install torch_sparse

In [ ]:
import sys
sys.path.append('/content/PyGCL')

Graph Classification Tasks

In [3]:
# import sys
# sys.path.append('/content/PyGCL')
import torch
import os.path as osp
import GCL.losses as L
import GCL.augmentors as A
import torch.nn.functional as F

from torch import nn
from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, SVMEvaluator
from GCL.models import DualBranchContrast
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset,Planetoid


def make_gin_conv(input_dim, out_dim):
    return GINConv(nn.Sequential(nn.Linear(input_dim, out_dim), nn.ReLU(), nn.Linear(out_dim, out_dim)))


class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(GConv, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                self.layers.append(make_gin_conv(input_dim, hidden_dim))
            else:
                self.layers.append(make_gin_conv(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        project_dim = hidden_dim * num_layers
        self.project = torch.nn.Sequential(
            nn.Linear(project_dim, project_dim),
            nn.ReLU(inplace=True),
            nn.Linear(project_dim, project_dim))

    def forward(self, x, edge_index, batch):
        z = x
        zs = []
        for conv, bn in zip(self.layers, self.batch_norms):
            z = conv(z, edge_index)
            z = F.relu(z)
            z = bn(z)
            zs.append(z)
        gs = [global_add_pool(z, batch) for z in zs]
        z, g = [torch.cat(x, dim=1) for x in [zs, gs]]
        return z, g


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, batch):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z, g = self.encoder(x, edge_index, batch)
        z1, g1 = self.encoder(x1, edge_index1, batch)
        z2, g2 = self.encoder(x2, edge_index2, batch)
        return z, g, z1, z2, g1, g2


def train(encoder_model, contrast_model, dataloader, optimizer):
    encoder_model.train()
    epoch_loss = 0
    for data in dataloader:
        data = data.to('cuda')
        optimizer.zero_grad()

        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)

        _, _, _, _, g1, g2 = encoder_model(data.x, data.edge_index, data.batch)
        g1, g2 = [encoder_model.encoder.project(g) for g in [g1, g2]]
        loss = contrast_model(g1=g1, g2=g2, batch=data.batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss


def test(encoder_model, dataloader):
    encoder_model.eval()
    x = []
    y = []
    for data in dataloader:
        data = data.to('cuda')
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        _, g, _, _, _, _ = encoder_model(data.x, data.edge_index, data.batch)
        x.append(g)
        y.append(data.y)
    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)

    split = get_split(num_samples=x.size()[0], train_ratio=0.6, test_ratio=0.2)
    result = SVMEvaluator(linear=True)(x, y, split)
    return result

Example Codes for Graph Classification

In [ ]:
def graph_classification():
    device = torch.device('cuda')
    path = osp.join(osp.expanduser('~'), 'datasets')
    dataset = TUDataset(path, name='PTC_MR')
    dataloader = DataLoader(dataset, batch_size=128)
    input_dim = max(dataset.num_features, 1)

    aug1 = A.Identity()
    aug2 = A.RandomChoice([A.RWSampling(num_seeds=1000, walk_length=10),
                           A.NodeDropping(pn=0.1),
                           A.FeatureMasking(pf=0.1),
                           A.EdgeRemoving(pe=0.1)], 1)
    gconv = GConv(input_dim=input_dim, hidden_dim=32, num_layers=2).to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2)).to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='G2G').to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.00001)
    test_results = []

    with tqdm(total=100, desc='(T)') as pbar:
        for epoch in range(1, 101):
            loss = train(encoder_model, contrast_model, dataloader, optimizer)
            if epoch % 5 == 0:
                test_result = test(encoder_model, dataloader)
                test_results.append([epoch,
                                     test_result['micro_f1'],
                                     test_result['macro_f1'],
                                     loss])
            # pbar.set_postfix({'loss': loss})
            # pbar.update()
    return test_results

    # test_result = test(encoder_model, dataloader)
    # print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')


if __name__ == '__main__':
    test_results = graph_classification()


Modify the codes from MVGL_node.py and GrapgCL.py to make it suitable for both the node-level and graph-level tasks.

In [8]:
import torch
import os.path as osp
import GCL.losses as L
import GCL.augmentors as A
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch import nn
from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, SVMEvaluator
from GCL.models import DualBranchContrast
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset,Planetoid


def make_gin_conv(input_dim, out_dim):
    return GINConv(nn.Sequential(nn.Linear(input_dim, out_dim), nn.ReLU(), nn.Linear(out_dim, out_dim)))


class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(GConv, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                self.layers.append(make_gin_conv(input_dim, hidden_dim))
            else:
                self.layers.append(make_gin_conv(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        project_dim = hidden_dim * num_layers
        self.project = torch.nn.Sequential(
            nn.Linear(project_dim, project_dim),
            nn.ReLU(inplace=True),
            nn.Linear(project_dim, project_dim))

    def forward(self, x, edge_index, batch):
        z = x
        zs = []
        for conv, bn in zip(self.layers, self.batch_norms):
            z = conv(z, edge_index)
            z = F.relu(z)
            z = bn(z)
            zs.append(z)
        z = torch.cat(zs, dim=1)
        z = self.project(z)
        return z


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, batch):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z = self.encoder(x, edge_index, batch)
        z1 = self.encoder(x1, edge_index1, batch)
        z2 = self.encoder(x2, edge_index2, batch)
        return z, z1, z2

def train_node(encoder_model, contrast_model, data, optimizer):
  encoder_model.train()
  optimizer.zero_grad()
  z, z1, z2 = encoder_model(data.x, data.edge_index, data.batch)
  loss = contrast_model(h1=z1, h2=z2, batch=data.batch)
  loss.backward()
  optimizer.step()
  return loss.item()

def test_node(encoder_model, data):
  encoder_model.eval()
  z, _, _ = encoder_model(data.x, data.edge_index, data.batch)
  split = get_split(num_samples=z.size()[0], train_ratio=0.6, test_ratio=0.2)
  result = SVMEvaluator(linear=True)(z, data.y, split)
  return result

def node_classification(option):
    device = torch.device('cuda')
    path = osp.join(osp.expanduser('~'), 'datasets')
    dataset = Planetoid(path, name='Cora', transform=T.NormalizeFeatures())
    data = dataset[0].to(device)
    input_dim = max(dataset.num_features, 1)

    ## Choose for different sparsification methods for augmentation ##
    # aug1 = A.Identity()
    aug1 = A.PPRDiffusion(alpha = 0.2)
    if option == 'fosr':
      aug2 = A.FOSR(max_iterations = 20)
    elif option == 'proxydelmin':
      aug2 = A.PROXYDELMIN(max_iterations = 20, seed = 4438)
    elif option == 'proxydelmax':
      aug2 = A.PROXYDELMAX(max_iterations = 20, seed = 4438)
    elif option == 'sdrf':
      aug2 = A.SDRF(max_iterations = 20)
    else:
      aug2 = A.Identity()
    gconv = GConv(input_dim=input_dim, hidden_dim=128, num_layers=2).to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2)).to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L').to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.0001)
    test_results = []

    with tqdm(total=100, desc='(T)') as pbar:
        for epoch in range(1, 101):
            loss = train_node(encoder_model, contrast_model, data, optimizer)
            if epoch % 5 == 0:
                test_result = test_node(encoder_model, data)
                test_results.append([epoch,
                                     test_result['micro_f1'],
                                     test_result['macro_f1'],
                                     loss])
            pbar.set_postfix({'loss': loss})
            pbar.update()
    return test_results


In [10]:
res5 = node_classification('other')

(T):   0%|          | 0/100 [00:00<?, ?it/s]

Shape before sparsification:  torch.Size([2, 10556])
Shape after sparsification:  torch.Size([2, 809194])


(T):   0%|          | 0/100 [05:20<?, ?it/s]


In [ ]:
res1 = node_classification('fosr')
res2 = node_classification('proxydelmin')
res3 = node_classification('proxydelmax')
res4 = node_classification('sdrf')
res5 = node_classification()

In [11]:
res5

[[5, 0.7182320441988951, 0.6868183972957068, 7.058711528778076],
 [10, 0.7090239410681399, 0.6887574527555437, 6.650815963745117],
 [15, 0.7255985267034991, 0.7033139699312111, 6.327315330505371],
 [20, 0.7845303867403315, 0.7617964628637377, 6.06370735168457],
 [25, 0.7753222836095764, 0.7788753362063533, 5.844958305358887],
 [30, 0.7863720073664825, 0.7719404999731175, 5.668851852416992],
 [35, 0.7900552486187845, 0.7806837406659869, 5.522538185119629],
 [40, 0.7753222836095764, 0.7545187088062921, 5.397488594055176],
 [45, 0.8103130755064457, 0.795856378688493, 5.288409233093262],
 [50, 0.8084714548802947, 0.7991368260196252, 5.195537567138672],
 [55, 0.8066298342541437, 0.7934009188143969, 5.107982635498047],
 [60, 0.7845303867403315, 0.7710376601492533, 5.031121730804443],
 [65, 0.8084714548802947, 0.790445760344556, 5.007458686828613],
 [70, 0.7808471454880295, 0.7644048329915439, 4.9321088790893555],
 [75, 0.8066298342541437, 0.8064882920759484, 4.86513614654541],
 [80, 0.799263